In [3]:
!pip install rouge

In [118]:
import numpy as np
import pandas as pd
import time

# Importing the T5 modules from huggingface/transformers
# T5ForConditionalGeneration is specific for sequence-to-sequence

#from nlp import load_metric
from rouge import Rouge


import glob, os
#import xml.etree.ElementTree as et
from lxml import etree as et
from collections import OrderedDict, defaultdict
import matplotlib.pyplot as plt
import numpy as np
import csv
import re

In [22]:
ami_test_set = [
    'ES2004a',
    'ES2004b',
    'ES2004c',
    'ES2004d',
    'ES2014a',
    'ES2014b',
    'ES2014c',
    'ES2014d',
    'IS1009a',
    'IS1009b',
    'IS1009c',
    'IS1009d',
    'TS3003a',
    'TS3003b',
    'TS3003c',
    'TS3003d',
    'TS3007a',
    'TS3007b',
    'TS3007c',
    'TS3007d'
]

icsi_test_set = [
    'Bed004',
    'Bed009',
    'Bed016',
    'Bmr005',
    'Bmr019',
    'Bro018'
]


In [30]:
len(ami_test_set)+len(icsi_test_set)

26

In [98]:
ROOT_DIR_BERT = "/Users/haileywu/Desktop/W266_project/BERTSUMresult/"

In [99]:
DIR_ICSI = "/Users/haileywu/Desktop/W266_project/data/ICSI_plus_NXT/"
DIR_AMI = "/Users/haileywu/Desktop/W266_project/data/AMI_manual/"

In [100]:
abs_sum_icsi = pd.read_csv(DIR_ICSI + "T5_csv/goldsummary_ICSI_as_test.csv")
abs_sum_ami = pd.read_csv(DIR_AMI + "T5_csv/goldsummary_AMI_as_test.csv")


In [103]:
abs_sum_icsi

,meeting,abstractive
0,Bed004,A test run of the data collection design was v...
1,Bed009,The Berkeley Even Deeper Understanding group d...
2,Bed016,The meeting was taken up by discussion about a...
3,Bmr005,Topics discussed by the Berkeley Meeting Recor...
4,Bmr019,The Berkeley Meeting Recorder group discussed ...
5,Bro018,The ICSI Meeting Recorder Group met once more ...


In [107]:
for prediction in sorted(glob.glob(ROOT_DIR_BERT+"TPRED_ami*.txt")):
    print(prediction.split("/")[-1].split(".")[0].split("_")[2])

ES2004a
ES2004b
ES2004c
ES2004d
ES2014a
ES2014b
ES2014c
ES2014d
IS1009a
IS1009b
IS1009c
IS1009d
TS3003a
TS3003b
TS3003c
TS3003d
TS3007a
TS3007b
TS3007c
TS3007d


In [112]:
def load_filler_words(path):
    with open(path, 'r+') as f:
        filler = f.read().splitlines()

    return filler

In [113]:
def clean_utterance(utterance, filler_words):
    utt = utterance
    # replace consecutive unigrams with a single instance
    utt = re.sub('\\b(\\w+)\\s+\\1\\b', '\\1', utt)
    # same for bigrams
    utt = re.sub('(\\b.+?\\b)\\1\\b', '\\1', utt)
    # strip extra white space
    utt = re.sub(' +', ' ', utt)
    # strip leading and trailing white space
    utt = utt.strip()

    # remove filler words # highly time-consuming
    utt = ' ' + utt + ' '
    for filler_word in filler_words:
        utt = re.sub(' ' + filler_word + ' '+ '.'+ ' ', ' ', utt)
        utt = re.sub(' ' + filler_word + ' '+ ','+ ' ', ' ', utt)
        utt = re.sub(' ' + filler_word + ' '+ '?'+ ' ', ' ', utt)
        utt = re.sub(' ' + filler_word + ' ', ' ', utt) 
        #utt = re.sub(' ' + filler_word.capitalize() + ' ', ' ', utt)

    return utt

In [114]:

def read_ami_icsi(diag, filler_words):
#     asr_output = pd.read_csv(
#         path,
#         sep='\t',
#         header=None,
#         names=['ID', 'start', 'end', 'letter', 'role', 'A', 'B', 'C', 'utt']
#     )
    utt = diag
    utterances = []
#     for tmp in zip(asr_output['role'].tolist(), asr_output['utt'].tolist()):
#         role, utt = tmp
#         # lower case
    utt = str(utt).lower()

    # remove special tag
    for ch in ['{nonvocalsound}','{vocalsound}', '{gap}', '{disfmarker}', '{comment}', '{pause}', '@reject@']:
        utt = re.sub(ch, '', utt)

    utt = re.sub("'Kay", 'okay', utt)
    utt = re.sub("'kay", 'okay', utt)
    utt = re.sub('"Okay"', 'okay', utt)
    utt = re.sub("'cause", 'cause', utt)
    utt = re.sub("'Cause", 'cause', utt)
    utt = re.sub('"cause"', 'cause', utt)
    utt = re.sub('"\'em"', 'them', utt)
    utt = re.sub('"\'til"', 'until', utt)
    utt = re.sub('"\'s"', 's', utt)
    utt = re.sub('"\"', "" , utt)
    utt = re.sub("-", ' ', utt)
    # l. c. d. -> lcd
    # t. v. -> tv
    utt = re.sub('h. t. m. l.', 'html', utt)
    utt = re.sub(r"(\w)\. (\w)\. (\w)\.", r"\1\2\3", utt)
    utt = re.sub(r"(\w)\. (\w)\.", r"\1\2", utt)
    utt = re.sub(r"(\w)\.", r"\1", utt)

    # clean_utterance, remove filler_words
    utt = clean_utterance(utt, filler_words=filler_words)

    # strip extra white space
    utt = re.sub(' +', ' ', utt)
    # strip leading and trailing white space
    utt = utt.strip()

    if not re.match(r'^[_\W]+$', utt) and utt != '':
    #if utt != '' and utt != '.' and utt != ' ' and utt!= "?" and utt!= "," and :
        utterances.append(utt)
    if len(utterances)>0:
    # remove duplicate utterances per speaker
        utterances = sorted(set(utterances), key=utterances.index)[0]
        result = str(utterances)
    else:
        result = ""
    #utterances_indexed = list(zip(range(len(utterances)), list(zip(*utterances))[0], list(zip(*utterances))[1]))
    #list(zip(*utterances))[1]
    return result

In [115]:
ROOT_DIR_UTILS = "/Users/haileywu/Desktop/W266_project/data/utils/"
filler_words = load_filler_words(ROOT_DIR_UTILS+"filler_words.less.txt")
#stopwords = load_stopwords(ROOT_DIR_UTILS+"stopwords.en.dat")

In [116]:
def clean_comma(utterance_indexed):
    #utterances_processed = []
#    for utterance_indexed in rst:
        # remove the comma at the beginning
    if len(utterance_indexed) >0:
        if utterance_indexed[0] == "," or utterance_indexed[0] == "." :
            utt_cleaned = utterance_indexed[2:]  
        else:
            utt_cleaned = utterance_indexed
        #print(utt_cleaned)
        #utterances_processed.append(utt_cleaned)
    else:
        utt_cleaned = ""
    return utt_cleaned

In [137]:
with open("../data/BERSUM/ICSI_test_cleaned.csv", mode='w') as csv_file:
    # ctext is the orginal text, while text is the extractive summary
    fieldnames = ['meeting','extractive','abstractive']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    
    for prediction in sorted(glob.glob(ROOT_DIR_BERT+"TPRED_icsi*.txt")):
        i=0
        text = ""
        paragraphcount = 0
        meetingname = prediction.split("/")[-1].split(".")[0].split("_")[2]
        if meetingname in icsi_test_set:
            abstractive_summary = abs_sum_icsi[abs_sum_icsi["meeting"]==meetingname]['abstractive'].values[0]
            file = open(prediction,mode='r')
            for f in file.readlines():
                diag = f.split("\n")[0] 
                diag = read_ami_icsi(diag, filler_words)
                diag = clean_comma(diag)
                diag_word_count = len(diag.split())
                if diag_word_count + paragraphcount > 1024:
                    writer.writerow({'meeting':meetingname+'.'+str(i), 'extractive': text, 'abstractive': abstractive_summary})
                    text = ""
                    paragraphcount = 0
                    i+=1
                text += diag
                paragraphcount += diag_word_count
            writer.writerow({'meeting':meetingname+'.'+str(i), 'extractive': text, 'abstractive': abstractive_summary})

In [138]:
df = pd.read_csv("../data/BERSUM/ICSI_test_cleaned.csv")

In [139]:
df[df['abstractive'].notna()]['extractive'].apply(lambda x: x.split(" ")).map(lambda x: len(x))

0     927
1     355
2     928
3     497
4     899
5     195
6     929
7     914
8     667
9     938
10    910
11    398
12    936
13    593
Name: extractive, dtype: int64